In [3]:
!pip -q install huggingface_hub langchain langchain_community huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
import torch
from huggingface_hub import HfApi
import os
from langchain.llms import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "..."
import re
import json

In [15]:
summarizer_prompt_template = """
You are a helpful AI assistant who is provided with a query, context and some previous chats with the user.
You have to predict whether the context and the previous memory retrieved by the retriever will be helpful while answering the giving query.
Craft a summarised version of the entire context and memory, focusing only the important information that you believe might prove useful while 
answering the user's query.

The query is:
{question}

Retrieved Context : {context}
Previous chats : {prev_chats}
"""

prompt_template = """
Generate 2 search queries that can be used to find information relevant to the given question.
Question: {question}.
Return the output in JSON format
Search queries: 
"""


In [29]:
class pipeline():
  def __init__(self, model):
    self.model = model

  def predict(self, prompt):
    llm = HuggingFaceHub(
    repo_id=self.model,
    model_kwargs={
        "temperature": 1.0,
        # "max_length": self.max_length,
        "max_new_tokens": 512,
    }
    )
    answer = llm(prompt)
    return answer

class assistant_model(pipeline):
  def __init__(self, model, prompt_template, summarizer_prompt_template):
    super().__init__(model,)
    self.prompt_template = prompt_template
    self.summarizer_prompt_template = summarizer_prompt_template

  def QA_decomposition(self, query):
      formatted_query = self.prompt_template.format(question=query)
      raw_answer = self.predict(formatted_query)
      match = re.search(r"({[^}]*})", raw_answer, re.DOTALL)
      if match:
          json_output = match.group(1).strip()
          result = json_output
      else:
          result = raw_answer.strip()
      result = json.loads(result)
      result["query3"] = query
      
      return result
        
  def summarizer(self, query, context, prev_chats):
    formatted_query = self.summarizer_prompt_template.format(question=query, context = context, prev_chats = prev_chats)
    raw_answer = self.predict(formatted_query)
    match = re.search(r"(1\..+)\n(2\..+)", raw_answer, re.DOTALL)
    if match:
        query1 = match.group(1).strip()
        query2 = match.group(2).strip()
        return f"{query}\n{query1}\n{query2}"
    else:
        return raw_answer.strip()
    

In [30]:
# model = "meta-llama/Llama-3.2-3B-Instruct"
model = "mistralai/Mistral-7B-Instruct-v0.3"
query = "Who won the football word cup 2022"

assistant = assistant_model(model, prompt_template, summarizer_prompt_template, )

In [31]:
result = assistant.QA_decomposition(query)
print(result)

{'query1': '2022 football world cup winner', 'query2': '2022 FIFA world cup champion', 'query3': 'Who won the football word cup 2022'}
